In [ ]:
#pip install resemblyzer

     |████████████████████████████████| 15.7 MB 8.6 MB/s 
     |████████████████████████████████| 78 kB 6.1 MB/s 
     |████████████████████████████████| 66 kB 4.5 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72309 sha256=df41808ec0839f6b2b742d54a5e2ef445b9a94bed8dcf900333766d3b4b4cea5
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26325 sha256=7edd4ebd363fda24dfe7a0e7634d204b17d884a6c20b56643016129db432fb33
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built webrtcvad typing


In [137]:
#importations
import numpy as np
import matplotlib.pyplot as plt
import resemblyzer as rz

In [138]:
from pathlib import Path
from resemblyzer import VoiceEncoder, preprocess_wav

In [139]:
from os import listdir
from os.path import isfile,join

In [140]:
import os

In [141]:
liste_chemins= [f for f in listdir("/") if ".flac" in f]

In [142]:
liste_chemins[:10]

['1246-124548-0000.flac',
 '6019-3185-0000.flac',
 '4788-294466-0000.flac',
 '6454-107462-0000.flac',
 '322-124146-0000.flac',
 '1578-140045-0000.flac',
 '3259-158083-0000.flac',
 '5390-24512-0000.flac',
 '8770-295462-0000.flac',
 '8088-284756-0000.flac']

In [143]:
#exemple avec le 1er fichier audio
wav = preprocess_wav("/"+liste_chemins[0])

encoder = VoiceEncoder()
embed = encoder.embed_utterance(wav)
np.set_printoptions(precision=3, suppress=True)
print(embed) #donne suite de nombres à partir de l'audio

Loaded the voice encoder model on cpu in 0.01 seconds.
[0.003 0.003 0.    0.    0.011 0.041 0.    0.049 0.    0.    0.157 0.163
 0.04  0.    0.    0.04  0.155 0.045 0.    0.    0.    0.094 0.    0.
 0.    0.077 0.003 0.    0.    0.    0.062 0.042 0.084 0.    0.03  0.017
 0.004 0.    0.    0.01  0.    0.046 0.033 0.235 0.022 0.083 0.    0.025
 0.032 0.06  0.    0.004 0.    0.    0.    0.001 0.006 0.    0.12  0.
 0.085 0.    0.256 0.    0.011 0.    0.054 0.    0.    0.    0.    0.02
 0.    0.074 0.    0.    0.    0.    0.008 0.04  0.002 0.    0.041 0.017
 0.    0.029 0.    0.126 0.    0.05  0.046 0.    0.    0.    0.141 0.
 0.004 0.    0.    0.    0.    0.    0.101 0.129 0.    0.077 0.    0.067
 0.    0.104 0.    0.069 0.08  0.185 0.003 0.    0.006 0.    0.    0.
 0.035 0.047 0.003 0.105 0.    0.128 0.    0.062 0.037 0.    0.156 0.
 0.    0.026 0.137 0.011 0.105 0.    0.005 0.004 0.    0.    0.002 0.
 0.066 0.    0.008 0.    0.112 0.    0.071 0.056 0.    0.    0.002 0.005
 0.    0.188 0.

In [146]:
from sklearn.linear_model import LogisticRegression 
from resemblyzer import preprocess_wav, VoiceEncoder
#from demo_utils import *
from pathlib import Path
from tqdm import tqdm

In [ ]:
## Gather a single utterance per speaker
data_dir = Path("audio_data", "librispeech_train-clean-100")
wav_fpaths = list(data_dir.glob("*.flac"))
speakers = [fpath.stem.split("-")[0] for fpath in wav_fpaths]
wavs = [preprocess_wav(wav_fpath) for wav_fpath in \
        tqdm(wav_fpaths, "Preprocessing wavs", len(wav_fpaths), unit=" utterances")]

# Get the sex of each speaker from the metadata file
with data_dir.joinpath("SPEAKERS.TXT").open("r") as f:
    sexes = dict(l.replace(" ", "").split("|")[:2] for l in f if not l.startswith(";"))
markers = ["x" if sexes[speaker] == "M" else "o" for speaker in speakers]
colors = ["black"] * len(speakers)


## Compute the embeddings
encoder = VoiceEncoder()
utterance_embeds = np.array(list(map(encoder.embed_utterance, wavs)))


## Project the embeddings in 2D space. 
_, ax = plt.subplots(figsize=(6, 6))
# Passing min_dist=1 to UMAP will make it so the projections don't necessarily need to fit in 
# clusters, so that you can have a better idea of what the manifold really looks like. 
projs = plot_projections(utterance_embeds, speakers, ax, colors, markers, False,
                         min_dist=1)
ax.set_title("Embeddings for %d speakers" % (len(speakers)))
ax.scatter([], [], marker="x", c="black", label="Male speaker")
ax.scatter([], [], marker="o", c="black", label="Female speaker")

# Separate the data by the sex
classifier = LogisticRegression(solver="lbfgs")
classifier.fit(projs, markers)
x = np.linspace(*ax.get_xlim(), num=200)
y = -(classifier.coef_[0, 0] * x + classifier.intercept_) / classifier.coef_[0, 1]
mask = (y > ax.get_ylim()[0]) & (y < ax.get_ylim()[1])
ax.plot(x[mask], y[mask], label="Decision boundary")

ax.legend()
plt.show()

In [147]:
wavs = [preprocess_wav("/"+wav_fpath) for wav_fpath in liste_chemins] #on récupère les listes de chiffres associées aux audios

In [180]:
dico={}
for element in liste_chemins :
  #dico[element]=(str(element).split("-")[0])
  #print(str(element).split("-")[0])
  dico[element]=sexes[str(element).split("-")[0]]

In [179]:
dico

{'103-1240-0000.flac': 'F',
 '1034-121119-0000.flac': 'M',
 '1040-133433-0000.flac': 'M',
 '1069-133699-0000.flac': 'F',
 '1081-125237-0000.flac': 'M',
 '1088-129236-0000.flac': 'F',
 '1098-133695-0000.flac': 'F',
 '1116-132847-0000.flac': 'F',
 '118-121721-0000.flac': 'M',
 '1183-124566-0000.flac': 'F',
 '1235-135883-0000.flac': 'M',
 '1246-124548-0000.flac': 'F',
 '125-121124-0000.flac': 'F',
 '1263-138246-0000.flac': 'F',
 '1334-135589-0000.flac': 'M',
 '1355-39947-0000.flac': 'M',
 '1363-135842-0000.flac': 'F',
 '1447-130550-0000.flac': 'F',
 '1455-134435-0000.flac': 'M',
 '150-126107-0000.flac': 'F',
 '1502-122615-0000.flac': 'F',
 '1553-140047-0000.flac': 'F',
 '1578-140045-0000.flac': 'F',
 '1594-135914-0000.flac': 'M',
 '1624-142933-0000.flac': 'M',
 '163-121908-0000.flac': 'M',
 '1723-141149-0000.flac': 'M',
 '1737-142396-0000.flac': 'F',
 '1743-142912-0000.flac': 'M',
 '1841-150351-0000.flac': 'F',
 '1867-148436-0000.flac': 'M',
 '1898-145702-0000.flac': 'F',
 '19-198-0000.fl

In [171]:
liste_sexe=list(sexes.values())
liste_sexe[:10] #OK

['F', 'F', 'M', 'F', 'F', 'F', 'F', 'M', 'M', 'M']

In [181]:
## Compute the embeddings
encoder = VoiceEncoder()
utterance_embeds = np.array(list(map(encoder.embed_utterance, wavs)))

Loaded the voice encoder model on cpu in 0.02 seconds.


In [184]:
utterance_embeds #on a une liste avec tous les audios sous forme de chiffres

array([[0.003, 0.003, 0.   , ..., 0.   , 0.   , 0.   ],
       [0.06 , 0.   , 0.037, ..., 0.   , 0.03 , 0.   ],
       [0.009, 0.   , 0.205, ..., 0.   , 0.131, 0.   ],
       ...,
       [0.025, 0.003, 0.021, ..., 0.   , 0.013, 0.   ],
       [0.   , 0.049, 0.   , ..., 0.   , 0.054, 0.   ],
       [0.148, 0.   , 0.126, ..., 0.   , 0.   , 0.   ]], dtype=float32)

In [185]:
with Path("/SPEAKERS.TXT").joinpath("/SPEAKERS.TXT").open("r") as f:
    sexes = dict(l.replace(" ", "").split("|")[:2] for l in f if not l.startswith(";"))
#markers = ["x" if sexes[speaker] == "M" else "o" for speaker in speakers]
#colors = ["black"] * len(speakers)

In [193]:
liste_id=[]
for element in liste_chemins:
  #print(str(element).split("-")[0])
  liste_id.append(str(element).split("-")[0])
#liste_chemins

In [204]:
df= pd.DataFrame(list(zip(liste_chemins,liste_id)), columns={"id","chemins"})

In [210]:
df=df.sort_values("id")[["chemins","id"]]
df

,chemins,id
0,103-1240-0000.flac,103
1,1034-121119-0000.flac,1034
2,1040-133433-0000.flac,1040
3,1069-133699-0000.flac,1069
4,1081-125237-0000.flac,1081
...,...,...
246,887-123289-0000.flac,887
247,89-218-0000.flac,89
248,8975-270782-0000.flac,8975
249,909-131041-0000.flac,909


In [212]:
df["id"]=df["id"].apply(lambda x : int(x))
df=df.sort_values("id")
df

,chemins,id
32,19-198-0000.flac,19
62,26-495-0000.flac,26
64,27-123349-0000.flac,27
81,32-21625-0000.flac,32
104,39-121914-0000.flac,39
...,...,...
242,8747-293952-0000.flac,8747
243,8770-295462-0000.flac,8770
244,8797-294123-0000.flac,8797
245,8838-298545-0000.flac,8838


In [229]:
df_gender=pd.DataFrame(list(sexes.items()), columns=["id","gender"])
df_gender["id"]=df_gender["id"].apply(lambda x : int(x))
df_gender

,id,gender
0,14,F
1,16,F
2,17,M
3,19,F
4,20,F
...,...,...
2479,8975,F
2480,9000,M
2481,9022,F
2482,9023,F


In [231]:
df=df.merge(df_gender, "left", on="id")  #on obtient un df avec l'id, le chemin d'accès et le gender
df

,chemins,id,gender
0,19-198-0000.flac,19,F
1,26-495-0000.flac,26,M
2,27-123349-0000.flac,27,M
3,32-21625-0000.flac,32,F
4,39-121914-0000.flac,39,F
...,...,...,...
246,8747-293952-0000.flac,8747,M
247,8770-295462-0000.flac,8770,M
248,8797-294123-0000.flac,8797,M
249,8838-298545-0000.flac,8838,M


In [233]:
#wavs = [preprocess_wav("/"+wav_fpath) for wav_fpath in liste_chemins] #on récupère les listes de chiffres associées aux audios
wavs = [preprocess_wav("/"+wav_fpath) for wav_fpath in df["chemins"]]

In [235]:
encoder = VoiceEncoder()
utterance_embeds = np.array(list(map(encoder.embed_utterance, wavs)))
utterance_embeds

Loaded the voice encoder model on cpu in 0.05 seconds.


array([[0.077, 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.06 , ..., 0.002, 0.023, 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.02 , 0.   ],
       ...,
       [0.065, 0.003, 0.077, ..., 0.   , 0.029, 0.   ],
       [0.072, 0.004, 0.014, ..., 0.   , 0.108, 0.   ],
       [0.   , 0.   , 0.003, ..., 0.   , 0.   , 0.   ]], dtype=float32)

In [242]:
df["utterance_embeds"]=list(utterance_embeds)
df

In [288]:
len(df["utterance_embeds"][120]) #les embeds ont la même taille

256

In [289]:
#on crée la classification binaire à la main -> reg logistique

df["male"]=df["gender"].apply(lambda x : int(x=="M"))
df

,chemins,id,gender,utterance_embeds,male
0,19-198-0000.flac,19,F,"[0.077119276, 0.0, 0.0, 0.0, 0.0, 0.040358894,...",0
1,26-495-0000.flac,26,M,"[0.0, 0.0, 0.060088374, 0.0, 0.0, 0.000605393,...",1
2,27-123349-0000.flac,27,M,"[0.0, 0.0, 0.0, 0.0, 0.0022264416, 0.012610136...",1
3,32-21625-0000.flac,32,F,"[0.0, 0.0154374875, 0.0058837747, 0.0, 0.01233...",0
4,39-121914-0000.flac,39,F,"[0.0, 0.05991597, 0.004216486, 0.0, 0.0, 0.105...",0
...,...,...,...,...,...
246,8747-293952-0000.flac,8747,M,"[0.0, 0.0, 0.16669533, 0.0, 0.005796113, 0.007...",1
247,8770-295462-0000.flac,8770,M,"[0.12583393, 0.0, 0.025904704, 0.0, 0.03144644...",1
248,8797-294123-0000.flac,8797,M,"[0.06472035, 0.0032126561, 0.07674578, 0.0, 0....",1
249,8838-298545-0000.flac,8838,M,"[0.07230248, 0.00400005, 0.014480232, 0.0, 0.0...",1


In [ ]:
#il faut transformer utterance_embeds en liste de variables

In [290]:
from sklearn.linear_model import LogisticRegression


In [306]:
#on va créer 256 features pour pouvoir faire la régression

df.head()

,chemins,id,gender,utterance_embeds,male
0,19-198-0000.flac,19,F,"[0.077119276, 0.0, 0.0, 0.0, 0.0, 0.040358894,...",0
1,26-495-0000.flac,26,M,"[0.0, 0.0, 0.060088374, 0.0, 0.0, 0.000605393,...",1
2,27-123349-0000.flac,27,M,"[0.0, 0.0, 0.0, 0.0, 0.0022264416, 0.012610136...",1
3,32-21625-0000.flac,32,F,"[0.0, 0.0154374875, 0.0058837747, 0.0, 0.01233...",0
4,39-121914-0000.flac,39,F,"[0.0, 0.05991597, 0.004216486, 0.0, 0.0, 0.105...",0


In [310]:
#création des features à partir de la liste utterance_embeds -> on passe par une boucle
x=[0 for element in range(256)]
for i in range(256):
  x[i]=[]

In [311]:
for element in df["utterance_embeds"]:
  for i in range(256):
    x[i].append(element[i])

In [324]:
df_features = pd.DataFrame(np.array(x).T, columns=columns_names)
df_features

In [321]:
columns_names=[f"x{i}" for i in range(256)]

In [330]:
df_features["id"]=df["id"]

In [333]:
df_with_features = pd.merge(df, df_features, on="id")
df_with_features.head()

,chemins,id,gender,utterance_embeds,male,x0,x1,x2,x3,x4,...,x246,x247,x248,x249,x250,x251,x252,x253,x254,x255
0,19-198-0000.flac,19,F,"[0.077119276, 0.0, 0.0, 0.0, 0.0, 0.040358894,...",0,0.077119,0.000000,0.000000,0.0,0.000000,...,0.086737,0.04361,0.082279,0.020890,0.083103,0.000000,0.027763,0.000000,0.000000,0.0
1,26-495-0000.flac,26,M,"[0.0, 0.0, 0.060088374, 0.0, 0.0, 0.000605393,...",1,0.000000,0.000000,0.060088,0.0,0.000000,...,0.025092,0.00000,0.151707,0.104390,0.000000,0.014027,0.009469,0.001877,0.023427,0.0
2,27-123349-0000.flac,27,M,"[0.0, 0.0, 0.0, 0.0, 0.0022264416, 0.012610136...",1,0.000000,0.000000,0.000000,0.0,0.002226,...,0.056097,0.00000,0.116862,0.148990,0.000000,0.014307,0.019038,0.000000,0.020239,0.0
3,32-21625-0000.flac,32,F,"[0.0, 0.0154374875, 0.0058837747, 0.0, 0.01233...",0,0.000000,0.015437,0.005884,0.0,0.012332,...,0.032443,0.00000,0.109765,0.177748,0.000000,0.028537,0.005702,0.000000,0.000000,0.0
4,39-121914-0000.flac,39,F,"[0.0, 0.05991597, 0.004216486, 0.0, 0.0, 0.105...",0,0.000000,0.059916,0.004216,0.0,0.000000,...,0.044643,0.00000,0.058638,0.116332,0.000000,0.011780,0.097484,0.000000,0.000000,0.0


In [345]:
import sklearn.model_selection

In [399]:
# regression logistique

X=df_with_features.iloc[:,5:]
y=df_with_features["male"]

modele_logit = LogisticRegression(penalty='none',solver='newton-cg')


In [431]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

In [432]:
modele_logit.fit(X_train,y_train)

LogisticRegression(penalty='none', solver='newton-cg')

In [389]:
def erreur(results, donnees) : 
  nbre_erreurs=0
  for i in range(len(results)):
    if results[i]!=donnees[i]:
      nbre_erreurs+=1
  #print(nbre_erreurs)
  train_error=nbre_erreurs/len(results)
  return train_error

#print("Erreur d'entrainement : ",100*train_error,"%")

In [433]:
results_train=modele_logit.predict(X_train)
results_test=modele_logit.predict(X_test)

In [434]:
print("Erreur d'entrainement: ",100*erreur(results_train, np.array(y_train)),"%")
print("Erreur de test: ",100*erreur(results_test, np.array(y_test)),"%")

Erreur d'entrainement:  0.0 %
Erreur de test:  1.9607843137254901 %


In [402]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [416]:
#validation croisée

def my_cross_val(X, y, modele, nbre_iter):
  compteur=0
  liste_scores=[]
  while compteur<nbre_iter :
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)
    modele.fit(X_train,y_train)
    clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
    liste_scores.append(clf.score(X_test, y_test))
    compteur+=1
  return liste_scores
  print("score moyen :",liste_scores.mean())

array([0.976, 1.   , 1.   , 1.   , 1.   , 1.   ])

In [447]:
my_cross_val(X, y, modele_logit, 5)

[1.0, 0.9803921568627451, 1.0, 1.0, 1.0]

In [ ]:
# prédiction à partir d'un audio